## Generator Expressions

similar to listcomp, but using () not []

In [1]:
symbols = '$¢£¥€¤'
tuple(ord(symbol) for symbol in symbols)

(36, 162, 163, 165, 8364, 164)

In [5]:
import array
array.array("I", (ord(symbol) for symbol in symbols))

array('I', [36, 162, 163, 165, 8364, 164])

In [6]:
colors = ['black', 'white']
sizes = ['S', 'M', 'L']
for tshirts in ("%s %s" % (c, s) for c in colors for s in sizes):
    print(tshirts)

black S
black M
black L
white S
white M
white L


## Tuple as record

*作为左值则是收集，作为函数调用则是解包

In [11]:
lax_coordinates = (33.9425, -118.408056)
city, *t, area = ('Tokyo', 2003, 32340, 0.66, 8014)
traveler_ids = [('USA', '31195855'), ('BRA', 'CE342567'), ('ESP', 'XDA205856')]
print(t)
for passport in sorted(traveler_ids):
    print("%s/%s" % passport)

[2003, 32340, 0.66]
BRA/CE342567
ESP/XDA205856
USA/31195855


In [8]:
t = (20, 8)
divmod(*t)

(2, 4)

In [15]:
a, b, *rest = range(5)
print(a, b, rest)

0 1 [2, 3, 4]


In [13]:
import os
_, filename = os.path.split("/hone/mory/python/hello.py")
print(filename)

hello.py


### Named Tuple 
+ saving space, just like struct in Clang
+ data can be passed as K-V pairs or positions
+ can be nested

In [18]:
from collections import namedtuple
city = namedtuple('City', 'name country population coordinates')
tokyo = city(name="Tokyo", country="JP", population=36.933, coordinates=(35.66, 136.77))
print(tokyo)

print(tokyo.population)
print(tokyo.coordinates)

City(name='Tokyo', country='JP', population=36.933, coordinates=(35.66, 136.77))
36.933
(35.66, 136.77)


In [22]:
# continue from previous example
print(city._fields)
# Nested Named tuple
LatLong = namedtuple("LatLong", 'lat long')
delhi_data = ('Delhi NCR', "IN", 21.935, LatLong(28.613889, 77.208889))
delhi = city._make(delhi_data)
delhi._asdict()

('name', 'country', 'population', 'coordinates')


OrderedDict([('name', 'Delhi NCR'),
             ('country', 'IN'),
             ('population', 21.935),
             ('coordinates', LatLong(lat=28.613889, long=77.208889))])

In [23]:
for key, value in delhi._asdict().items():
    print(key + ":", value)

name: Delhi NCR
country: IN
population: 21.935
coordinates: LatLong(lat=28.613889, long=77.208889)


### Tuples as Immutable Lists
tuple support almost every methods list support except those modify function

## Slicing
+ name the slice!

In [31]:
invoice = """
0.....6.................................40........52...55........
1909  Pimoroni PiBrella                     $17.50    3 $52.50
1489  6mm Tactile Switch x20                $4.95     2 $9.90
1510  Panavise Jr. - PV-201                 $28.00    1 $28.00
1601  PiTFT Mini Kit 320x240                $34.95    1 $34.95
"""
SKU = slice(0, 6)
DESCRIPTION = slice(6, 40)
UNIT_PRICE = slice(40, 52)
QUANTITY = slice(52, 55)
ITEM_TOTAL = slice(55, None)
line_items = invoice.split("\n")[2:]
for item in line_items:
    print(item[UNIT_PRICE], item[DESCRIPTION])

    $17.50   Pimoroni PiBrella                 
    $4.95    6mm Tactile Switch x20            
    $28.00   Panavise Jr. - PV-201             
    $34.95   PiTFT Mini Kit 320x240            
 


In [35]:
import numpy as np
array = np.array([[[1,1,1], [2,2,2], [3,3,3]],
                 [[1,1,1], [2,2,2], [3,3,3]],
                 ])
array[1, ...]

array([[1, 1, 1],
       [2, 2, 2],
       [3, 3, 3]])

In [36]:
t = (1, 2, [30, 40])
t[2] += [50, 60]

TypeError: 'tuple' object does not support item assignment

## Managing Ordered Sequences with bisect

In [47]:
import bisect
import sys

HAYSTACK = [1, 4, 5, 6, 8, 12, 15, 20, 21, 23, 23, 26, 29, 30]
NEEDLES = [0, 1, 2, 5, 8, 10, 22, 23, 29, 30, 31]

ROW_FMT = '{0:2d} @ {1:2d}    {2}{0:<2d}'

def demo(bisect_fn):
    for needle in reversed(NEEDLES):
        position = bisect_fn(HAYSTACK, needle)
        offset = position * "  |"
        print(ROW_FMT.format(needle, position, offset))
        
bisect_fn = bisect.bisect
print("DEMO:", bisect_fn.__name__)
print("haystack ->", " ".join("%2d" % n for n in HAYSTACK))
demo(bisect_fn)


DEMO: bisect
haystack ->  1  4  5  6  8 12 15 20 21 23 23 26 29 30
31 @ 14      |  |  |  |  |  |  |  |  |  |  |  |  |  |31
30 @ 14      |  |  |  |  |  |  |  |  |  |  |  |  |  |30
29 @ 13      |  |  |  |  |  |  |  |  |  |  |  |  |29
23 @ 11      |  |  |  |  |  |  |  |  |  |  |23
22 @  9      |  |  |  |  |  |  |  |  |22
10 @  5      |  |  |  |  |10
 8 @  5      |  |  |  |  |8 
 5 @  3      |  |  |5 
 2 @  1      |2 
 1 @  1      |1 
 0 @  0    0 


### inspire! using bisect to covert test score to letter grades

In [48]:
def grade(score, breakpoints=[60, 70, 80, 90], grades="FDCBA"):
    i = bisect.bisect(breakpoints, score)
    return grades[i]

[grade(score) for score in [33, 99, 45, 88, 77, 66, 100]]

['F', 'A', 'F', 'B', 'C', 'D', 'A']

### Inserting with bisect.insort

In [49]:
import bisect
import random

SIZE = 7
random.seed(1729)

my_list = []
for i in range(SIZE):
    new_item = random.randrange(SIZE*2)
    bisect.insort(my_list, new_item)
    print("%2d ->" % new_item, my_list)

10 -> [10]
 0 -> [0, 10]
 6 -> [0, 6, 10]
 8 -> [0, 6, 8, 10]
 7 -> [0, 6, 7, 8, 10]
 2 -> [0, 2, 6, 7, 8, 10]
10 -> [0, 2, 6, 7, 8, 10, 10]
